### 线性回归从零开始
生成一个包含1000个样本的数据集， 每个样本包含从标准正态分布中采样的2个特征。 合成数据集是一个矩阵， $\mathbf{X}\in \mathbb{R}^{1000 \times 2}$
使用线性模型参数 $\mathbf{w} = [2, -3.4]^\top$ 和噪声项e生成数据集及其标签 $\mathbf{y}= \mathbf{X} \mathbf{w} + b + \mathbf\epsilon.$


In [2]:
import torch

In [4]:
def synthetic_data(w, b, num_examples):  #@save
    """生成y=Xw+b+噪声"""
    X = torch.normal(mean=0, std=1, size=(num_examples, len(w))) # 从标准正态分步采样1000,2的矩阵
    y = torch.matmul(X, w) + b      # 矩阵相乘。 y = X * w + b
    y += torch.normal(0, 0.01, y.shape) # 模拟误差
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)
print(f'{features.shape}, {labels.shape}')
print('features:', features[0],'\nlabel:', labels[0])

torch.Size([1000, 2]), torch.Size([1000, 1])
features: tensor([-0.8154, -1.3041]) 
label: tensor([7.0020])
